In [1]:
import os
import argparse
from solver import Solver
from data_loader import get_loader
from torch.backends import cudnn
import numpy as np
from datetime import datetime
import json

def str2bool(v):
    return v.lower() in ('true')

def main(config):
    # For fast training.
    cudnn.benchmark = True

    # Create directories if not exist.
    if not os.path.exists(config.log_dir):
        os.makedirs(config.log_dir)
    if not os.path.exists(config.model_save_dir):
        os.makedirs(config.model_save_dir)
    if not os.path.exists(config.sample_dir):
        os.makedirs(config.sample_dir)
    if not os.path.exists(config.result_dir):
        os.makedirs(config.result_dir)

    # Data loader.
    celeba_loader = None
    rafd_loader = None

    if config.dataset in ['CelebA', 'Both']:
        celeba_loader = get_loader(config.celeba_image_dir, config.attr_path, config.selected_attrs,
                                   config.celeba_crop_size, config.image_size, config.batch_size,
                                   'CelebA', config.mode, config.num_workers)
    if config.dataset in ['RaFD', 'Both']:
        rafd_loader = get_loader(config.rafd_image_dir, None, None,
                                 config.rafd_crop_size, config.image_size, config.batch_size,
                                 'RaFD', config.mode, config.num_workers)
    

    # Solver for training and testing StarGAN.
    solver = Solver(celeba_loader, rafd_loader, config)

    if config.mode == 'train':
        if config.dataset in ['CelebA', 'RaFD']:
            solver.train()
        elif config.dataset in ['Both']:
            solver.train_multi()
    elif config.mode == 'test':
        if config.dataset in ['CelebA', 'RaFD']:
            solver.test()
        elif config.dataset in ['Both']:
            solver.test_multi()

In [2]:
class Config():
    """Params"""
    def random_G_lr(self):
        return np.random.uniform(10**-3, 10**-5)

    def random_D_lr(self):
        return np.random.uniform(10**-3, 10**-5)

    def random_n_critic(self):
        return int(np.random.randint(1, 11, size=1)[0])
    
    def random_Adam_beta1(self):
        return np.random.uniform(0.5, 0.9)
    
    def dir_name(self):
        return 'stargan/' + datetime.now().strftime("%m%d-%H:%M:%S")
    
    def dump(self):
        if not os.path.exists(self.dir):
            os.makedirs(self.dir)
        with open(self.dir +'/config.json','w') as f:
            json.dump({'g_lr': self.g_lr, 'd_lr': self.d_lr, 'n_critic': self.n_critic, 'Adam_beta1': self.beta1, 'selected_attrs': self.selected_attrs}, fp=f, sort_keys=True, indent=4)
        print(json.dumps({'g_lr': self.g_lr, 'd_lr': self.d_lr, 'n_critic': self.n_critic, 'Adam_beta1': self.beta1, 'selected_attrs': self.selected_attrs}, sort_keys=True, indent=4))

    def __init__(self):
        # Model configuration.
        self.c_dim = 5 #adjust for selected attributes
        self.c2_dim = 8
        self.celeba_crop_size = 178
        self.rafd_crop_size=256
        self.image_size = 128
        self.g_conv_dim = 64
        self.d_conv_dim = 64
        self.g_repeat_num = 6
        self.d_repeat_num = 6
        self.lambda_cls = 1
        self.lambda_rec = 10
        self.lambda_gp = 10
        
        # Training configuration.
        self.dataset = 'CelebA'
        self.batch_size = 16
        self.num_iters = 20000
        self.num_iters_decay = 20000
        self.g_lr = self.random_G_lr()
        self.d_lr = self.random_D_lr()
        self.n_critic = self.random_n_critic()
        self.beta1 = self.random_Adam_beta1()
        self.beta2 = 0.999
        self.resume_iters = None
        self.selected_attrs = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips',
            'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby',
            'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male',
            'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose',
            'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
            'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young'
            ]
        self.c_dim = len(self.selected_attrs)
        
        # Test configuration.
        self.test_iters = 20000
        
        # Miscellaneous.
        self.num_workers = 1
        self.mode = 'train'
        self.use_tensorboard = True
        
        # Directories.
        self.celeba_image_dir = 'data/CelebA_nocrop/images'
        self.attr_path = 'data/list_attr_celeba.txt'
        self.rafd_image_dir = 'data/RaFD/train'
        self.dir = self.dir_name()
        self.log_dir = self.dir +'/logs'
        self.model_save_dir = self.dir + '/models'
        self.sample_dir = self.dir + '/samples'
        self.result_dir = self.dir + '/results'
        
        #Step size
        self.log_step = 10
        self.sample_step = 1000
        self.model_save_step = 10000
        self.lr_update_step = 1000
        
        self.dump()

In [3]:
config = Config()

{
    "Adam_beta1": 0.8412060106311363,
    "d_lr": 0.00039881362401106226,
    "g_lr": 0.00017279763335057352,
    "n_critic": 6,
    "selected_attrs": [
        "5_o_Clock_Shadow",
        "Arched_Eyebrows",
        "Attractive",
        "Bags_Under_Eyes",
        "Bald",
        "Bangs",
        "Big_Lips",
        "Big_Nose",
        "Black_Hair",
        "Blond_Hair",
        "Blurry",
        "Brown_Hair",
        "Bushy_Eyebrows",
        "Chubby",
        "Double_Chin",
        "Eyeglasses",
        "Goatee",
        "Gray_Hair",
        "Heavy_Makeup",
        "High_Cheekbones",
        "Male",
        "Mouth_Slightly_Open",
        "Mustache",
        "Narrow_Eyes",
        "No_Beard",
        "Oval_Face",
        "Pale_Skin",
        "Pointy_Nose",
        "Receding_Hairline",
        "Rosy_Cheeks",
        "Sideburns",
        "Smiling",
        "Straight_Hair",
        "Wavy_Hair",
        "Wearing_Earrings",
        "Wearing_Hat",
        "Wearing_Lipstick",
        "Wear

In [ ]:
main(config)

Finished preprocessing the CelebA dataset...
Generator(
  (main): Sequential(
    (0): Conv2d(43, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ResidualBlock(
      (main): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(256, 256, kernel_size=(3

/home/paperspace/.pyenv/versions/anaconda3-5.2.0/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Elapsed [0:00:03], Iteration [10/20000], D/loss_gp: 0.1742, D/loss_fake: 2.6247, D/loss_cls: 18.0657, D/loss_real: -5.2827
Elapsed [0:00:06], Iteration [20/20000], D/loss_gp: 0.1148, D/loss_fake: -0.8097, D/loss_cls: 20.0472, D/loss_real: -15.2547
Elapsed [0:00:09], Iteration [30/20000], D/loss_gp: 0.5548, D/loss_cls: 24.6008, G/loss_rec: 0.4473, D/loss_real: -36.6399, G/loss_cls: 31.6630, G/loss_fake: 7.1860, D/loss_fake: 3.1525
Elapsed [0:00:11], Iteration [40/20000], D/loss_gp: 1.3533, D/loss_fake: 13.0501, D/loss_cls: 22.6091, D/loss_real: -56.2784
Elapsed [0:00:14], Iteration [50/20000], D/loss_gp: 0.2143, D/loss_fake: 5.8615, D/loss_cls: 22.0538, D/loss_real: -31.5923
Elapsed [0:00:17], Iteration [60/20000], D/loss_gp: 0.2950, D/loss_cls: 17.9168, G/loss_rec: 0.3887, D/loss_real: -31.3555, G/loss_cls: 22.4623, G/loss_fake: -12.4109, D/loss_fake: 8.2200
Elapsed [0:00:20], Iteration [70/20000], D/loss_gp: 0.2878, D/loss_fake: 3.0479, D/loss_cls: 17.2136, D/loss_real: -30.1921
Elaps